## Huggingface GRPOTrainer with Accelerate Multi-GPU Training on Amazon SageMaker

In [1]:
import os
from dotenv import load_dotenv
import boto3
from huggingface_hub import HfFolder
from sagemaker.modules import Session
from sagemaker.modules.train import ModelTrainer
from sagemaker.modules.distributed import Torchrun
from sagemaker.modules.train.model_trainer import Mode
from sagemaker.modules.configs import (
    Compute,
    SourceCode,
    InputData,
    CheckpointConfig,
)


load_dotenv()  # wandb and/or huggingface token in .env file

region = 'eu-central-1'
boto_session = boto3.Session(region_name=region)

/home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/19/25 01:43:14] INFO     Found credentials from IAM Role:                                   ]8;id=233609;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=960578;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/botocore/credentials.py#1132\1132]8;;\
                             AmazonSSMRoleForInstancesQuickSetup                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


In [2]:
pytorch_image = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu126-ubuntu22.04-sagemaker'
# define the script to be run
source_code = SourceCode(
    source_dir="scripts/",
    requirements="requirements.txt",
    # entry_script="dpo.py",
    command="accelerate launch --config_file /opt/ml/input/data/code/default_config.yaml /opt/ml/input/data/code/grpo.py",
)

environment = {
    'HF_TOKEN': HfFolder.get_token(),
    'WANDB_API_KEY': os.environ.get('WANDB_API_KEY'),
}

compute = Compute(
    instance_count=1,
    instance_type="ml.p4d.24xlarge",
    # volume_size_in_gb=96,
    keep_alive_period_in_seconds=3600,
)

In [3]:
sess = Session(boto_session=boto_session)
bucket = sess.default_bucket()
base_job_name = "grpo-trl"
checkpoint_path = f"s3://{bucket}/{base_job_name}/checkpoints/"

# define the ModelTrainer
model_trainer = ModelTrainer(
    sagemaker_session=sess,
    training_image=pytorch_image,
    source_code=source_code,
    base_job_name=base_job_name,
    compute=compute,
    environment=environment,
    checkpoint_config=CheckpointConfig(
        s3_uri=checkpoint_path,
    ),
)

[04/19/25 01:43:15] INFO     Found credentials from IAM Role:                                   ]8;id=35403;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=913875;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/botocore/credentials.py#1132\1132]8;;\
                             AmazonSSMRoleForInstancesQuickSetup                                                   

[04/19/25 01:43:18] WARNING  Role not provided. Using default role:                            ]8;id=215014;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=577377;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#487\487]8;;\
                             arn:aws:iam::364430515305:role/AmazonSSMRoleForInstancesQuickSetu                     
                             p                                                                                     

                    WARNING  StoppingCondition not provided. Using default:                    ]8;id=313567;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=839700;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#518\518]8;;\
                             max_runtime_in_seconds=3600 max_wait_time_in_seconds=None                             
                             max_pending_time_in_seconds=None                                                      

                    WARNING  OutputDataConfig not provided. Using default:                     ]8;id=339087;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=189078;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#563\563]8;;\
                             s3_output_path='s3://sagemaker-eu-central-1-364430515305/grpo-trl                     
                             ' kms_key_id=None compression_type='GZIP'                                             

                    INFO     Training image URI:                                               ]8;id=338981;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=842792;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#569\569]8;;\
                             763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:                     
                             2.6.0-gpu-py312-cu126-ubuntu22.04-sagemaker                                           

In [4]:
# start the training job
model_trainer.train(wait=False)

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=546072;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=131054;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

RESOURCE_CONFIG: volume_size_in_gb=30 instance_type='ml.p4d.24xlarge' instance_count=1 volume_kms_key_id=None keep_alive_period_in_seconds=3600 instance_groups=None training_plan_arn=None
VPC_CONFIG: None
SESSION: Session(region_name='eu-central-1')


[04/19/25 01:43:23] INFO     Creating training_job resource.                                     ]8;id=874448;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=362557;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/resources.py#28140\28140]8;;\

                    WARNING  No region provided. Using default region.                                 ]8;id=954701;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=23103;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/utils.py#340\340]8;;\

                    WARNING  No config provided. Using default config.                                 ]8;id=732213;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/utils.py\utils.py]8;;\:]8;id=416647;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker_core/main/utils.py#344\344]8;;\

[04/19/25 01:43:24] WARNING  Not displaing the training container logs as 'wait' is set to     ]8;id=431091;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py\model_trainer.py]8;;\:]8;id=856036;file:///home/ubuntu/rl-on-sagemaker/.venv/lib/python3.12/site-packages/sagemaker/modules/train/model_trainer.py#742\742]8;;\
                             False.                                                                                

In [ ]:
# With command line arguments

# source_code = SourceCode(
#     source_dir="scripts/",
#     requirements="requirements.txt",
#     # entry_script="grpo_advanced.py",
#     command="accelerate launch --config_file /opt/ml/input/data/code/default_config.yaml /opt/ml/input/data/code/grpo_advanced.py \
#             --dataset_name trl-lib/tldr \
#             --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
#             --reward_model_name_or_path Qwen/Qwen2-72B-Instruct-AWQ \
#             --output_dir Qwen2-0.5B-GRPO-ADV \
#             --report_to wandb",
# )